# Otimização com o Algoritmo de Seleção Clonal (CLONALG)

## 1. Introdução

### Função Rastrigin - Exemplo de Otimização Contínua

Neste notebook, aplicamos o ClonAlg à **Função Rastrigin**, um problema clássico de otimização contínua 
usando algoritmos de otimização do pacote AISP.

A função Rastrigin é uma função multimodal não-convexa que possui muitos mínimos locais, 
tornando-a um excelente teste para algoritmos de otimização. A função é definida como:

**f(x) = 10n + Σ[i=1 to n] (x_i² - 10cos(2πx_i))**

Onde:
- **n** é a dimensão do problema
- **x_i** ∈ [-5.12, 5.12] para cada dimensão
- **Mínimo global**: f(0,0) = 0

## 2. Configuração

### Instalação das bibliotecas necessários.

In [ ]:
%pip install matplotlib
%pip install aisp

### Importação das bibliotecas necessárias

In [ ]:
# Importando o Algoritmo de Seleção Clonal (CLONALG)
from aisp.csa import Clonalg

# Bibliotecas para manipulação de dados e cálculos numéricos
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt

## 3. Definição do problema

In [ ]:
# Configuração do problema
problem_size = 2
bounds = {'min': -5.12, 'max': 5.12}

def rastrigin_fitness(x: np.ndarray) -> np.float64:
    x = np.clip(x, bounds['min'], bounds['max'])
    
    n = len(x)
    result = 10 * n
    for i in range(n):
        result += x[i]**2 - 10 * np.cos(2 * np.pi * x[i])
    return np.float64(result)

## 4. Configuração do algoritmo

In [ ]:
# Configuração otimizada do ClonAlg para função Rastrigin
clonalg = Clonalg(
    problem_size=problem_size,
    N=50,
    selection_size=15,
    rate_clonal=10,
    rate_hypermutation=0.3,
    n_diversity_injection=15,
    bounds=bounds,
    seed=1234
)

# Registrar a função de fitness
clonalg.register('affinity_function', rastrigin_fitness)


## 5. Execução da otimização

In [ ]:
clonalg.optimize(100, 3)

if clonalg.best_cost is not None:
	print('Best cost:', abs(clonalg.best_cost))

## 6. Resultado

In [ ]:
print(clonalg.get_report())

### 6.1 Evolução do melhor ao longo das gerações

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Preparar dados da função Rastrigin (malha para visualização)
x1 = np.linspace(bounds['min'], bounds['max'], 200)
x2 = np.linspace(bounds['min'], bounds['max'], 200)
X, Y = np.meshgrid(x1, x2)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = rastrigin_fitness([X[i, j], Y[i, j]])

# Melhor solução do ClonAlg
best_solution = clonalg.best_solution
best_cost = clonalg.best_cost

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131)
costs = np.array(clonalg.cost_history)

ax1.plot(costs, marker='o', color='tab:green', label='Custo')

ax1.set_title('Evolução do Custo', fontsize=12)
ax1.set_xlabel('Iterações')
ax1.set_ylabel('Custo / Afinidade Minimizada')
ax1.grid(True, linestyle='--', alpha=0.5)
ax1.legend()

ax2 = fig.add_subplot(132)
contour = ax2.contour(X, Y, Z, levels=20, cmap='viridis')
if best_solution is not None:
    ax2.scatter(best_solution[0], best_solution[1], color='red', s=80, label=f'Melhor Solução\nf={best_cost:.4f}')
    ax2.legend()
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.grid(True, alpha=0.3)
ax2.set_title('Curvas de Nível')

ax3 = fig.add_subplot(133)
im = ax3.imshow(Z, extent=(bounds['min'], bounds['max'], bounds['min'], bounds['max']),
                origin='lower', cmap='viridis')
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')
ax3.set_title('Heatmap da Função')
plt.colorbar(im, ax=ax3, shrink=0.8)

# Marcar melhor solução no heatmap
if best_solution is not None:
    ax3.scatter(best_solution[0], best_solution[1], color='red', s=80, label=f'Melhor Solução\nf={best_cost:.4f}')
    ax3.legend()

fig.suptitle('Otimização da Função Rastrigin com ClonAlg', fontsize=16, fontweight='bold')
plt.show()
